In [1]:
import os

In [2]:
import detect

In [3]:
import numpy as np
import cv2
from sklearn.cluster import KMeans
from collections import Counter
import imutils
import pprint
from matplotlib import pyplot as plt
from PIL import Image
import os, sys

# 플라스크 서버 연동할 때 경로랑 손가락 구분하는거 다시 생각해보기
def img_mask_read(save_source, design_nail_crops):
    # 사용자 전체 손 사진
    img = cv2.imread(save_source)
    # 잘려진 손톱 사진
    mask = cv2.imread(design_nail_crops)
    return img, mask

def find_img_finger_1(img):
    dst = img[178:331, 247:354].copy()
    height, width, channel = dst.shape
    x = 247
    y = 178
    return dst, x, y, height, width

def find_img_finger_2(img):
    dst = img[226:381, 389:511].copy()
    height, width, channel = dst.shape
    x = 389
    y = 226
    return dst, x, y, height, width

def find_img_finger_3(img):
    dst = img[300:447, 542:669].copy()
    height, width, channel = dst.shape
    x = 542
    y = 300
    return dst, x, y, height, width

def find_img_finger_4(img):
    dst = img[386:503, 670:786].copy()
    height, width, channel = dst.shape
    x = 670
    y = 386
    return dst, x, y, height, width

def prety_print_data(color_info):
    for x in color_info:
        print(pprint.pformat(x))
        print()
        

        
def plotColorBar(colorInformation):
    # Create a 500x100 black image
    color_bar = np.zeros((100, 500, 3), dtype="uint8")

    top_x = 0
    for x in colorInformation:
        bottom_x = top_x + (x["color_percentage"] * color_bar.shape[1])

        color = tuple(map(int, (x['color'])))

        cv2.rectangle(color_bar, (int(top_x), 0),
                      (int(bottom_x), color_bar.shape[0]), color, -1)
        top_x = bottom_x
    return color_bar


def extractDominantColor(image, number_of_colors=5, hasThresholding=False):

    # Quick Fix Increase cluster counter to neglect the black(Read Article)
    if hasThresholding == True:
        number_of_colors += 1

    # Taking Copy of the image
    img = image.copy()

    # Convert Image into RGB Colours Space
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Reshape Image
    img = img.reshape((img.shape[0]*img.shape[1]), 3)

    # Initiate KMeans Object
    estimator = KMeans(n_clusters=number_of_colors, random_state=0)

    # Fit the image
    estimator.fit(img)

    # Get Colour Information
    colorInformation = getColorInformation(
        estimator.labels_, estimator.cluster_centers_, hasThresholding)
    return colorInformation

def getColorInformation(estimator_labels, estimator_cluster, hasThresholding=False):

    # Variable to keep count of the occurance of each color predicted
    occurance_counter = None

    # Output list variable to return
    colorInformation = []

    # Check for Black
    hasBlack = False

    # If a mask has be applied, remove th black
    if hasThresholding == True:

        (occurance, cluster, black) = removeBlack(
            estimator_labels, estimator_cluster)
        occurance_counter = occurance
        estimator_cluster = cluster
        hasBlack = black

    else:
        occurance_counter = Counter(estimator_labels)

    # Get the total sum of all the predicted occurances
    totalOccurance = sum(occurance_counter.values())

    # Loop through all the predicted colors
    for x in occurance_counter.most_common(len(estimator_cluster)):

        index = (int(x[0]))

        # Quick fix for index out of bound when there is no threshold
        index = (index-1) if ((hasThresholding & hasBlack)
                              & (int(index) != 0)) else index

        # Get the color number into a list
        color = estimator_cluster[index].tolist()

        # Get the percentage of each color
        color_percentage = (x[1]/totalOccurance)

        # make the dictionay of the information
        colorInfo = {"cluster_index": index, "color": color,
                     "color_percentage": color_percentage}

        # Add the dictionary to the list
        colorInformation.append(colorInfo)

    return colorInformation

def removeBlack(estimator_labels, estimator_cluster):

    # Check for black
    hasBlack = False

    # Get the total number of occurance for each color
    occurance_counter = Counter(estimator_labels)

    # Quick lambda function to compare to lists
    def compare(x, y): return Counter(x) == Counter(y)

    # Loop through the most common occuring color
    for x in occurance_counter.most_common(len(estimator_cluster)):

        # Quick List comprehension to convert each of RBG Numbers to int
        color = [int(i) for i in estimator_cluster[x[0]].tolist()]

        # Check if the color is [0,0,0] that if it is black
        if compare(color, [0, 0, 0]) == True:
            # delete the occurance
            del occurance_counter[x[0]]
            # remove the cluster
            hasBlack = True
            estimator_cluster = np.delete(estimator_cluster, x[0], 0)
            break

    return (occurance_counter, estimator_cluster, hasBlack)

def extractSkin(image):
    # Taking a copy of the image
    img = image.copy()
    # Converting from BGR Colours Space to HSV
    img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # Defining HSV Threadholds
    lower_threshold = np.array([0, 48, 80], dtype=np.uint8)
    upper_threshold = np.array([20, 255, 255], dtype=np.uint8)

    # Single Channel mask,denoting presence of colours in the about threshold
    skinMask = cv2.inRange(img, lower_threshold, upper_threshold)

    # Cleaning up mask using Gaussian Filter
    skinMask = cv2.GaussianBlur(skinMask, (3, 3), 0)

    # Extracting skin from the threshold mask
    skin = cv2.bitwise_and(img, img, mask=skinMask)

    # Return the Skin image
    return cv2.cvtColor(skin, cv2.COLOR_HSV2BGR)

# 손톱에서 피부 색만 제외한 부분 검은색으로 하기 
def find_design_nail(mask):
    nailBasic = imutils.resize(mask, width=250)
    # 손톱에서 피부색만 추출
    skin = extractSkin(nailBasic)
    dominantColors2 = extractDominantColor(skin, hasThresholding=True)
    prety_print_data(dominantColors2)
    # 추출해낸 피부색을 담고 있는 막대기
    colour_bar = plotColorBar(dominantColors2)
    return skin, mask

# skin 색상을 제외한 손톱의 검은 영역을 흰색으로 반전시키기 
def black_2_white(skin):
    # skin색상 부분만 그레이로 바꾼뒤 검은색 손톱 부분 제외하고 흰색으로 처리하기
    nailGray2 = cv2.cvtColor(skin, cv2.COLOR_BGR2GRAY)
    th2, mask2 = cv2.threshold(nailGray2, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    
    # 흰색부분이 마스크가 되기 위해서 검은색 손톱 부분을 흰색으로 반전
    mask_b_inv = cv2.bitwise_not(mask2)
    return mask_b_inv, mask2

def mask_extract(mask, mask_b_inv):
    height, width, channel = mask.shape
    mask_b_inv = cv2.resize(mask_b_inv, (width, height))
    
    #마스크에 해당하는 이미지만 추출하기
    img_fg = cv2.bitwise_and(mask, mask, mask=mask_b_inv)
    return img_fg

def rotate_resize_1(img_fg, height, width):
    # 회전하기
    img_fg_copy = img_fg.copy()
    h, w = img_fg_copy.shape[:2]
    
    nail = cv2.getRotationMatrix2D((w//2 , h//2 ), -20, 1)
    img_fg_rotate = cv2.warpAffine(img_fg_copy, nail,(w, h))
    
    # resize
    img_fg_resize = cv2.resize(img_fg_rotate, (width+20, height+10))
    return img_fg_resize

def rotate_resize_2(img_fg, height, width):
    # 회전하기
    img_fg_copy = img_fg.copy()
    h, w = img_fg_copy.shape[:2]
    
    nail = cv2.getRotationMatrix2D((w//2 , h//2 ), -20, 1)
    img_fg_rotate = cv2.warpAffine(img_fg_copy, nail,(w, h))
    
    
    # resize
    img_fg_resize = cv2.resize(img_fg_rotate, (width+5, height+10))
    return img_fg_resize

def rotate_resize_3(img_fg, height, width):
    # 회전하기
    img_fg_copy = img_fg.copy()
    h, w = img_fg_copy.shape[:2]
    
    nail = cv2.getRotationMatrix2D((w//2 , h//2 ), -10, 1)
    img_fg_rotate = cv2.warpAffine(img_fg_copy, nail,(w, h))
    
    
    # resize
    img_fg_resize = cv2.resize(img_fg_rotate, (width+20, height+25))
    return img_fg_resize

def rotate_resize_4(img_fg, height, width):
    # 회전하기
    img_fg_copy = img_fg.copy()
    h, w = img_fg_copy.shape[:2]
    
    nail = cv2.getRotationMatrix2D((w//2 , h//2 ), -25, 1)
    img_fg_rotate = cv2.warpAffine(img_fg_copy, nail,(w, h))
    
    
    # resize
    img_fg_resize = cv2.resize(img_fg_rotate, (width-4, height+36))
    return img_fg_resize

def transparent(img_fg_resize, x, y, save_source):
    color_coverted = cv2.cvtColor(img_fg_resize, cv2.COLOR_BGR2RGB)
    pillow_mask = Image.fromarray(color_coverted)
    pil_mask = pillow_mask.convert("RGBA")
    datas = pil_mask.getdata()
    
    newData = []
    for item in datas:
        if item[0] < 10 and item[1] < 10 and item[2] < 10:
            newData.append((item[0], item[1], item[2], 0))
        else:
            newData.append(item)
    
    pil_mask.putdata(newData)
    
    # 여기는 플라스크 서버 연동하고 경로 변경해야함
    pillow_img = Image.open(save_source)
    pillow_img.paste(pil_mask, (x, y), pil_mask)
    return pillow_img   

def transparent2(img_fg_resize, x, y,save_source):
    color_coverted = cv2.cvtColor(img_fg_resize, cv2.COLOR_BGR2RGB)
    pillow_mask = Image.fromarray(color_coverted)
    pil_mask = pillow_mask.convert("RGBA")
    datas = pil_mask.getdata()
    
    newData = []
    for item in datas:
        if item[0] < 10 and item[1] < 10 and item[2] < 10:
            newData.append((item[0], item[1], item[2], 0))
        else:
            newData.append(item)
    
    pil_mask.putdata(newData)
    
    # 여기는 플라스크 서버 연동하고 경로 변경해야함
    pillow_img = Image.open(save_source)
    pillow_img.paste(pil_mask, (x, y), pil_mask)
    return pillow_img    

In [4]:
from flask import Flask
from flask import request, redirect

app = Flask(__name__)

# 일반적인 라우트 방식입니다.
@app.route('/')
def board():
    return "hello world!!!"

@app.route('/predict', methods=['POST'])
def predict() :
    result={}
    if request.method == 'POST':
        
        print("플라스크 접속")
        # apache에서 img 받아오기 
        display(request.form)
        user_nail = request.files['user_nail']
        user_id = request.form['user_id']
        name = user_nail.filename
        name = name[:-4] + '_1' + name[-4:]
        
        
        # img를 source 경로에 저장
        save_source = "C:/Users/smhrd/git/thumbnail/Thumbnail/src/main/webapp/resources/image/"+user_nail.filename
        save_source1 = "C:/Users/smhrd/git/thumbnail/Thumbnail/src/main/webapp/resources/image/"+name

        user_nail.save(save_source)
        # 사용자가 업로드한 사진 저장 경로를 DB에 저장해야함
        
        # best 모델의 경로
        best_yolo = "./runs/train/v8t2test1437_high_epochs500/weights/best.pt"
        # design_nail_crops가 저장되어 있는 경로
        design_nail_crops1 = "./runs/detect/nail_design/crops/1/3.jpg"
        design_nail_crops2 = "./runs/detect/nail_design/crops/2/3.jpg"
        design_nail_crops3 = "./runs/detect/nail_design/crops/3/3.jpg"
        design_nail_crops4 = "./runs/detect/nail_design/crops/4/3.jpg"
        # 결과가 저장될 경로
        result_name = "result_" + user_nail.filename
        result_source = "C:/Users/smhrd/git/thumbnail/Thumbnail/src/main/webapp/resources/result_img/"+ result_name
        # 결과 사진이 저장된 경로를 DB에 저장해야함
        
        # 사용자의 손 사진 -> crop된 좌표가 필요함
        # user_nail_after = "./runs/detect/"+user_id+"/crops/"
        
        # yolo 돌리는 코드
        # detect.run(source="C:\Users\smhrd\git\thumbnail\Thumbnail\src\main\webapp\resources\image\")
        # 사용자에게 받아온 손 사진 yolo 돌리는 코드 : 손 사진은 프로젝트 파일 내의 경로
        # !python detect.py --source $save_source --weights $best_yolo --save-txt --name $user_id --save-crop
        
        img, mask1 = img_mask_read(save_source, design_nail_crops1)
        mask2 = cv2.imread(design_nail_crops2)
        mask3 = cv2.imread(design_nail_crops3)
        mask4 = cv2.imread(design_nail_crops4)
        print("img.shape : ", img.shape)
        print("mask.shape : ", mask1.shape)
        
        dst1, x1, y1, height1, width1 = find_img_finger_1(img)
        print("x1, y1, height1, width1 : ", x1, y1, height1, width1)
        
        dst2, x2, y2, height2, width2 = find_img_finger_2(img)
        print("x, y, height, width : ", x2, y2, height2, width2)
        
        dst3, x3, y3, height3, width3 = find_img_finger_3(img)
        print("x, y, height, width : ", x3, y3, height3, width3)
        
        dst4, x4, y4, height4, width4 = find_img_finger_4(img)
        print("x, y, height, width : ", x4, y4, height4, width4)
        
        skin1, mask1 = find_design_nail(mask1)
        mask_b_inv1, mask1_1 = black_2_white(skin1)
        print("mask_b_inv.shape : ", mask_b_inv1.shape)
        print("mask2.shape : ", mask1_1.shape)

        skin2, mask2 = find_design_nail(mask2)
        mask_b_inv2, mask2_1 = black_2_white(skin2)
        print("mask_b_inv.shape : ", mask_b_inv2.shape)
        print("mask2.shape : ", mask2_1.shape)

        skin3, mask3 = find_design_nail(mask3)
        mask_b_inv3, mask3_1 = black_2_white(skin3)
        print("mask_b_inv.shape : ", mask_b_inv3.shape)
        print("mask2.shape : ", mask3_1.shape)

        skin4, mask4 = find_design_nail(mask4)
        mask_b_inv4, mask4_1 = black_2_white(skin4)
        print("mask_b_inv.shape : ", mask_b_inv4.shape)
        print("mask2.shape : ", mask4_1.shape)

        
        img_fg1 = mask_extract(mask1, mask_b_inv1)
        print("img_fg.shape : ", img_fg1.shape)
        print("height, width : ", height1, width1)

        img_fg2 = mask_extract(mask2, mask_b_inv2)
        print("img_fg.shape : ", img_fg2.shape)
        print("height, width : ", height2, width2)

        img_fg3 = mask_extract(mask3, mask_b_inv3)
        print("img_fg.shape : ", img_fg3.shape)
        print("height, width : ", height3, width3)

        img_fg4 = mask_extract(mask4, mask_b_inv4)
        print("img_fg.shape : ", img_fg4.shape)
        print("height, width : ", height4, width4)

        
        img_fg_resize1 = rotate_resize_1(img_fg1, height1, width1)
        print("img_fg_resize.shape : ", img_fg_resize1.shape)

        img_fg_resize2 = rotate_resize_2(img_fg2, height2, width2)
        print("img_fg_resize.shape : ", img_fg_resize2.shape)

        img_fg_resize3 = rotate_resize_3(img_fg3, height3, width3)
        print("img_fg_resize.shape : ", img_fg_resize3.shape)

        img_fg_resize4 = rotate_resize_4(img_fg4, height4, width4)
        print("img_fg_resize.shape : ", img_fg_resize4.shape)
        
        pillow_image = transparent(img_fg_resize1, x1, y1, save_source)
        pillow_image = pillow_image.convert("RGB")
        pillow_image.save(save_source1)
        pillow_image = transparent2(img_fg_resize2, x2, y2, save_source1)
        pillow_image = pillow_image.convert("RGB")
        pillow_image.save(save_source1)
        pillow_image = transparent2(img_fg_resize3, x3, y3, save_source1)
        pillow_image = pillow_image.convert("RGB")
        pillow_image.save(save_source1)
        pillow_image = transparent2(img_fg_resize4, x4, y4, save_source1)
        pillow_image = pillow_image.convert("RGB")
        pillow_image.save(save_source1)
        pillow_image.save(result_source)
        
        
        url = 'http://localhost:8081/nail/single_product2.do?save_source='+save_source+'&result_source='+result_source
        return redirect(url)
        
# 호스트와 포트번호는 변경하지 마세요
# flask 접속 url : http://f1.project-jupyter.ddns.net:8871
if __name__ == '__main__':
    app.run(host="127.0.0.1",port='5000')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


플라스크 접속


ImmutableMultiDict([('user_id', 'saga1897@naver.com')])

127.0.0.1 - - [25/Apr/2022 18:29:03] "POST /predict HTTP/1.1" 400 -


...

127.0.0.1 - - [25/Apr/2022 18:33:01] "POST /predict HTTP/1.1" 302 -


플라스크 접속


ImmutableMultiDict([('user_id', '')])

img.shape :  (720, 960, 3)
mask.shape :  (161, 122, 3)
x1, y1, height1, width1 :  247 178 153 107
x, y, height, width :  389 226 155 122
x, y, height, width :  542 300 147 127
x, y, height, width :  670 386 117 116
{'cluster_index': 4,
 'color': [165.6368381804591, 97.09563758389227, 81.95469798657945],
 'color_percentage': 0.29975013881177126}

{'cluster_index': 2,
 'color': [144.22696392689525, 80.45383216916856, 69.35384173763357],
 'color_percentage': 0.2878956135480289}

{'cluster_index': 3,
 'color': [177.90949302483818, 123.85539299081108, 108.1247022796885],
 'color_percentage': 0.1641588006662965}

{'cluster_index': 1,
 'color': [116.11789562645257, 56.146207479400076, 47.65159518275935],
 'color_percentage': 0.1308717379233759}

{'cluster_index': 0,
 'color': [144.0368794326238, 98.1907801418426, 92.8070921985826],
 'color_percentage': 0.11732370905052748}

mask_b_inv.shape :  (329, 250)
mask2.shape :  (329, 250)
{'cluster_index': 4,
 'color': [146.3124124649873, 87.706582633

127.0.0.1 - - [26/Apr/2022 09:43:22] "POST /predict HTTP/1.1" 302 -


In [16]:
save_source = "C:/Users/smhrd/git/thumbnail/Thumbnail/src/main/webapp/resources/image/19.png"
design_nail_crops = "./runs/detect/nail_design/crops/1/3.jpg"

img, mask = img_mask_read(save_source, design_nail_crops)
print("img.shape : ", img.shape)
print("mask.shape : ", mask.shape)

img.shape :  (720, 960, 3)
mask.shape :  (161, 122, 3)


In [17]:
dst, x, y, height, width = find_img_finger_1(img)
print("x, y, height, width : ", x, y, height, width)
print(dst.shape)

x, y, height, width :  257 188 153 107
(153, 107, 3)


In [18]:
skin, mask = find_design_nail(mask)
mask_b_inv, mask2 = black_2_white(skin)
print("mask_b_inv.shape : ", mask_b_inv.shape)
print("mask2.shape : ", mask2.shape)

{'cluster_index': 4,
 'color': [165.6368381804591, 97.09563758389227, 81.95469798657945],
 'color_percentage': 0.29975013881177126}

{'cluster_index': 2,
 'color': [144.22696392689525, 80.45383216916856, 69.35384173763357],
 'color_percentage': 0.2878956135480289}

{'cluster_index': 3,
 'color': [177.90949302483816, 123.85539299081108, 108.1247022796885],
 'color_percentage': 0.1641588006662965}

{'cluster_index': 1,
 'color': [116.11789562645255, 56.146207479400076, 47.65159518275935],
 'color_percentage': 0.1308717379233759}

{'cluster_index': 0,
 'color': [144.0368794326238, 98.1907801418426, 92.80709219858258],
 'color_percentage': 0.11732370905052748}

mask_b_inv.shape :  (329, 250)
mask2.shape :  (329, 250)


In [19]:
img_fg = mask_extract(mask, mask_b_inv)
print("img_fg.shape : ", img_fg.shape)
print("height, width : ", height, width)

img_fg.shape :  (161, 122, 3)
height, width :  153 107


In [20]:
img_fg_resize=rotate_resize_1(img_fg, height, width)
print("img_fg_resize.shape : ", img_fg_resize.shape)

img_fg_resize.shape :  (163, 112, 3)


In [22]:
result_name = "result_" + "19.png"
result_source = "C:/Users/smhrd/git/thumbnail/Thumbnail/src/main/webapp/resources/result_img/"+ result_name
pillow_image = transparent(img_fg_resize, x, y, save_source)
pillow_image.save(result_source)